In [11]:
import numpy as np
import pandas as pd
import datetime as dt

from numpy.random import seed
from tensorflow import set_random_seed

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

# __Preprocessing__

In [12]:
# Read data and save as dataframe
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Create list of public holidays in Chicago (source: https://publicholidays.us/illinois/2018-dates/ and https://publicholidays.us/illinois/2019-dates/)
holidays = ['2018-10-08', '2018-11-06', '2018-11-11', '2018-11-12', '2018-11-22', '2018-11-23', '2018-12-25', 
            '2019-01-01', '2019-01-21', '2019-02-12', '2019-02-18', '2019-05-27', '2019-07-04', '2019-09-02']

# Split timestamps into date and time
df.trip_start_timestamp = pd.to_datetime(df['trip_start_timestamp'])
df_test.trip_start_timestamp = pd.to_datetime(df_test['trip_start_timestamp'])

# Add weekday to new column and determine workdays and rushhours
df['trip_start_dow'] = df.trip_start_timestamp.dt.dayofweek
df['is_workday'] = np.where((df.trip_start_dow < 5) & (~df.trip_start_timestamp.dt.date.isin(holidays)), 1, 0)
df['is_rushhour'] = np.where(
    (df.is_workday == 1) & 
    (((df.trip_start_timestamp.dt.time >= dt.time(7, 0, 0)) & (df.trip_start_timestamp.dt.time <= dt.time(9, 0, 0))) | 
      ((df.trip_start_timestamp.dt.time >= dt.time(16, 0, 0)) & (df.trip_start_timestamp.dt.time <= dt.time(18, 0, 0)))), 1, 0)

df_test['trip_start_dow'] = df_test.trip_start_timestamp.dt.dayofweek
df_test['is_workday'] = np.where((df_test.trip_start_dow < 5) & (~df_test.trip_start_timestamp.dt.date.isin(holidays)), 1, 0)

# Drop columns
columns = ['id', 'tolls', 'extras', 'tips', 'company', 'taxi_id', 'pickup_community_area', 'dropoff_community_area', 'trip_end_timestamp', 
           'pickup_centroid_longitude', 'pickup_centroid_latitude', 'dropoff_centroid_longitude', 'dropoff_centroid_latitude']
df.drop(columns, axis=1, inplace=True)
df_test.drop(columns, axis=1, inplace=True)

# Create date column and delete time and id columns
df.trip_start_timestamp = df.trip_start_timestamp.dt.date

# Pre-clean data
df.dropna(inplace=True)
df = df[df.trip_miles > 0]
df = df[df.trip_miles < 100]
df = df[df.trip_seconds > 0]
df = df[df.trip_seconds < 7200]
df = df[df.trip_total > 0]
df = df[df.trip_total < 100]

df_test.fare.fillna(df.fare.mean(), inplace=True)
df_test.trip_seconds.fillna(df.trip_seconds.mean(), inplace=True)
df_test.trip_miles.fillna(df.trip_miles.mean(), inplace=True)
df_test.trip_total.fillna(df.trip_total.mean(), inplace=True)
df_test.payment_type.fillna(0, inplace=True)

# Create additional columns
df['price_per_mile'] = df.fare / df.trip_miles
df = df[df.price_per_mile > 0]
df = df[df.price_per_mile < 10]
df['table_fare'] = 3.25 + 2.25 * df.trip_miles + df.trip_seconds / 36 * 0.20 # source: https://www.chicago.gov/city/en/depts/bacp/supp_info/2012_passenger_information.html
df['overpriced'] = np.where(df.fare > df.table_fare, 1, 0)
df.drop('table_fare', axis=1, inplace=True)

df_test['price_per_mile'] = np.where(df_test.trip_miles != 0, df_test.fare / df_test.trip_miles, 0)
df_test['table_fare'] = 3.25 + 2.25 * df_test.trip_miles + df_test.trip_seconds / 36 * 0.20 # source: https://www.chicago.gov/city/en/depts/bacp/supp_info/2012_passenger_information.html
df_test['overpriced'] = np.where(df_test.fare > df_test.table_fare, 1, 0)
df_test.drop('table_fare', axis=1, inplace=True)

# Drop columns
columns = ['trip_start_timestamp', 'fare', 'trip_start_dow']
df.drop(columns, axis=1, inplace=True)
df_test.drop(columns, axis=1, inplace=True)

# One-hot encode categorical data
df['payment_type'] = np.where(df.payment_type == 'Cash', 1, 0)
df_test['payment_type'] = np.where(df_test.payment_type == 'Cash', 1, 0)

# Normalize data (feature scaling)
df = (df - df.min()) / (df.max() - df.min())
df_test = (df_test - df_test.min()) / (df_test.max() - df_test.min())

# Determine training and test set
X_train = df.loc[:, df.columns != 'is_rushhour'].to_numpy()
y_train = df.loc[:, df.columns == 'is_rushhour'].to_numpy()
X_test = df_test.to_numpy()

# Reduce dimensions (PCA)
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# __Model__

In [13]:
# Parameters determined by previous hyperparameter tuning
def create_model(lyrs=[8], act='linear', opt='Adagrad', dr=0.5):
    
    seed(2020)
    set_random_seed(2020)
    
    model = Sequential()
    
    # create first hidden layer
    model.add(Dense(lyrs[0], input_dim=X_train.shape[1], activation=act))
    
    # create additional hidden layers
    for i in range(1, len(lyrs)):
        model.add(Dense(lyrs[i], activation=act))
    
    # add dropout, default is none
    model.add(Dropout(dr))
    
    # create output layer
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

# __Training__

In [14]:
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=64, verbose=1)
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, verbose=1)
y_pred = model.predict(X_test)

Train on 121268 samples, validate on 30318 samples
Epoch 1/100
121268/121268 [==============================] - 2s 20us/sample - loss: 0.6401 - acc: 0.6557 - val_loss: 0.6066 - val_acc: 0.7931
Epoch 2/100
121268/121268 [==============================] - 2s 18us/sample - loss: 0.5984 - acc: 0.7444 - val_loss: 0.5754 - val_acc: 0.7931
Epoch 3/100
121268/121268 [==============================] - 2s 18us/sample - loss: 0.5736 - acc: 0.7689 - val_loss: 0.5533 - val_acc: 0.7931
Epoch 4/100
121268/121268 [==============================] - 2s 18us/sample - loss: 0.5557 - acc: 0.7788 - val_loss: 0.5366 - val_acc: 0.7931
Epoch 5/100
121268/121268 [==============================] - 2s 19us/sample - loss: 0.5423 - acc: 0.7834 - val_loss: 0.5236 - val_acc: 0.7931
Epoch 6/100
121268/121268 [==============================] - 2s 20us/sample - loss: 0.5320 - acc: 0.7862 - val_loss: 0.5133 - val_acc: 0.7931
Epoch 7/100
121268/121268 [==============================] - 2s 20us/sample - loss: 0.5239 - acc:

# __Export__

In [16]:
# Extract predictions as CSV
result = pd.DataFrame(y_pred, columns=['prediction'])
result.prediction = result.prediction.astype(int)
result.insert(0, 'id', range(200001, 200001 + len(result)))
result.to_csv('~/prediction.csv', index=False)